# Le  package *boot*

Le package **boot** contient des fonctions  puissantes et optimisées pour le bootstrap. En effet, les fonctions que vous avez programmées au dernier notebook sont toutes disponibles dans le package **boot**. Cela ne veut pas dire que votre travail a été inutile. Au fait, la mise en oeuvre des différentes fonctions bootstrap vous a sûrement aidé a mieux comprendre les méthodes !

Maintenant chargons le package par l'instruction suivante :


In [1]:
library(boot)

### Répliques bootstrap

La fonction de base est la fonction **boot**. Elle génère des répliques bootstrap d'un estimateur $T$ - au choix par le bootstrap paramétrique ou non paramétrique. Elle a des nombreuses options :

Voyons d'abord le bootstrap nonparamétrique. Les arguments de **boot** sont les suivants : le vecteur des données (**data**), une fonction qui définit l'estimateur $T$ (**statistics**) et le nombre $B$ de répliques souhaités. Par exemple :

In [2]:
n <- 20
obs <- rnorm(n)
B <- 10
boot.mean <- function(obs, indice){
    moy.boot <- mean(obs[indice]) # permet à boot de choisir les données
    return(moy.boot)
}
repl.np <- boot(obs, boot.mean, B)
repl.np


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = obs, statistic = boot.mean, R = B)


Bootstrap Statistics :
     original       bias    std. error
t1* 0.1075842 -0.008344936   0.1378235

On obtient toute une liste d'objets en sortie :



In [3]:
names(repl.np)

[1] "t0"        "t"         "R"         "data"      "seed"      "statistic"
 [7] "sim"       "call"      "stype"     "strata"    "weights"

dont les plus utiles sont :
- repl\$t0 : la valeur $t=T(x_1,\dots,x_n)$ de l'estimateur sur les données observées
- repl\$t : le vecteur des répliques bootstrap de l'estimateur

In [4]:
repl.np$t

0.21297451
0.21905528
0.20151205
-0.04511531
0.02567735
-0.13200655
0.18235508
-0.01129438
0.05905018
0.28018485


In [5]:
repl.np$t0

[1] 0.1075842

Pour utiliser l'approche bootstrap paramétrique, c'est légèrement plus compliqué :
- il faut préciser  **sim='parametric'**,
- **statistic**  est une fonction qui prend en argument un vecteur de données et renvoie la valeur de l'estimateur sur ces données,
-  **ran.gen** est la  fonction qui génère un échantillon bootstrap paramétrique. Elle prend en argument les données et une liste de paramètres.
-  **mle** prend les valeurs observées des paramètres (que **boot** passera à la fonction **ran.gen**).

Dans le cas de la moyenne empirique comme estimateur et des échantillons  bootstrap tirés selon une loi normale, ça donne ça :

In [6]:
ran.gen.norm <- function(data, param){
  rnorm(length(data), param$mu, param$sig)
}
repl.param <- boot(obs, mean, B, sim='parametric', ran.gen=ran.gen.norm,
                   mle=list(mu=mean(obs), sig=sd(obs)))
repl.param


PARAMETRIC BOOTSTRAP


Call:
boot(data = obs, statistic = mean, R = B, sim = "parametric", 
    ran.gen = ran.gen.norm, mle = list(mu = mean(obs), sig = sd(obs)))


Bootstrap Statistics :
     original      bias    std. error
t1* 0.1075842 -0.02798379   0.1037511

## Question 1
Considérons, une fois de plus, un échantillon $(X_1,\dots,X_n)$ de loi exponentielle $\mathcal E(\theta)$. La quantité à estimer est $1/\mathbb E[X_1]$, que l'on estime naturellement par $T=1/\bar X_n$.
- Quelle est l'instruction pour créer 1000 répliques bootstrap non paramétrique de l'estimateur $T$ en utilisant la fonction **boot** ?
- A l'image de **ren.gen.norm**, écrire une fonction **ren.gen.exp** pour la génération d'échantillons bootstrap de loi exponentielle.
- Utiliser **boot** pour créer des répliques bootstrap paramétrique de $T$.

In [7]:
## répliques bootstrap non paramétrique
dataexp <- rexp(20, 3)
T.exp.np <- boot(dataexp, function(obs, indice)1/mean(obs[indice]), R=1000)
T.exp.np


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = dataexp, statistic = function(obs, indice) 1/mean(obs[indice]), 
    R = 1000)


Bootstrap Statistics :
    original    bias    std. error
t1* 2.834158 0.2121998   0.9447146

In [8]:
## génération d'un échnatillon bootstrap de loi exponentielle
ran.gen.exp <- function(data, param){
  rexp(length(data), param)
}

In [9]:
## répliques bootstrap paramétrique
T.exp.param <- boot(dataexp, function(x)1/mean(x), 1000, sim='parametric',
                    ran.gen=ran.gen.exp, mle=1/mean(dataexp))
T.exp.param


PARAMETRIC BOOTSTRAP


Call:
boot(data = dataexp, statistic = function(x) 1/mean(x), R = 1000, 
    sim = "parametric", ran.gen = ran.gen.exp, mle = 1/mean(dataexp))


Bootstrap Statistics :
    original    bias    std. error
t1* 2.834158 0.1826154    0.758384

### Intervalles de confiance

Pour le calcul de divers intervalles bootstrap, on utilise la fonction **boot.ci** :

Le premier argument est un objet obtenu par un appel à **boot** (essentiellement, ce sont des répliques bootstrap de l'estimateur). 

Avec **type** on précise le(s) intervalles bootstrap souhaité(s) : au choix **c("norm","basic", "stud", "perc", "bca")** ou simplement **type = "all"** (par défaut) pour tous les intervalles.


Voici l'appel le plus simple de **boot.ci** (qui produit un *warning* car l'intervalle studentisé nécessite plus d'information) pour calculer des intervalles de confiance bootstrap non paramétrique de la moyenne de la loi normale basés sur la moyenne empirique :

In [10]:
B <- 1000
repl.np <- boot(obs, boot.mean, B)
icb.np <- boot.ci(repl.np)
icb.np

Warning message in boot.ci(repl.np):
“bootstrap variances needed for studentized intervals”

BOOTSTRAP CONFIDENCE INTERVAL CALCULATIONS
Based on 1000 bootstrap replicates

CALL : 
boot.ci(boot.out = repl.np)

Intervals : 
Level      Normal              Basic         
95%   (-0.1789,  0.3976 )   (-0.1698,  0.3923 )  

Level     Percentile            BCa          
95%   (-0.1771,  0.3850 )   (-0.1706,  0.3979 )  
Calculations and Intervals on Original Scale

Au fait, l'instruction **boot.ci(repl.np)** calcule les quatre intervalles bootstrap suivants : l'intervalle par approximation normale $\mathcal I^*_{\text{norm}}$, l'intervalle de base $\mathcal I^*_{\text{basic}}$, l'intervalle par la méthode de percentile de base $\mathcal I^*_{\text{perc}}$ et l'intervalle par la méthode des percentiles ajustée $\mathcal I^*_{\mathrm{BC_a}}$. Plus précisément, **boot.ci** renvoie une liste avec les éléments suivants :

In [11]:
names(icb.np)

[1] "R"       "t0"      "call"    "normal"  "basic"   "percent" "bca"

Voici le niveau de confiance nominal $1-\alpha$ ainsi que les limites de l'intervalle $\mathcal I^*_{\text{norm}}$ :

In [12]:
icb.np$normal

conf,,
0.95,-0.1788912,0.3976094


Pour l'intervalle $\mathcal I^*_{\text{basic}}$ on a les informations suivantes : d'abord le niveau de confiance nominal, ensuite l'ordre utilisé des statistiques d'ordres dans les limites de l'intervalle, et enfin les limites de l'intervalle bootstrap :

In [13]:
icb.np$basic

conf,,,,
0.95,975.98,25.03,-0.1697914,0.3923089


Les mêmes informations pour l'intervalle $\mathcal I^*_{\text{perc}}$ :

In [14]:
icb.np$percent

conf,,,,
0.95,25.03,975.98,-0.1771405,0.3849599


et pour l'intervalle $\mathcal I^*_{\mathrm{BC_a}}$ :

In [15]:
icb.np$bca

conf,,,,
0.95,30.31,980.62,-0.1706117,0.3979474


Pour le calcul de l'intervalle bootstrap studentisé $\mathcal I^*_{\text{stud}}$ il faut fournir des estimations de la variance de $T$ pour les différents échantillons bootstrap. Une façon de faire consiste à modifier la fonction **boot.mean** pour qu'elle renvoie également l'estimateur de la variance :

In [16]:
obs <- rnorm(20,10,2)       
boot.mean.var <- function(obs, indice){
    obs.boot <- obs[indice] 
    moy.boot <- mean(obs.boot) 
    var.boot <- var(replicate(50, mean(sample(obs.boot, replace=T))))
    return(c(moy.boot, var.boot))
}
repl.np.var <- boot(obs, boot.mean.var, R=1000)
repl.np.var


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = obs, statistic = boot.mean.var, R = 1000)


Bootstrap Statistics :
      original       bias    std. error
t1* 10.2316292 -0.004789746  0.44971693
t2*  0.2324494 -0.047620961  0.06348872

Maintenat on obtient l'intervalle bootstrap studentisé par l'instruction suivante :

In [17]:
boot.ci(repl.np.var)$stud

conf,,,,
0.95,975.98,25.03,9.224323,11.4323


## Question 2
- Pour un jeu de données simulé de loi exponentielle $\mathcal E(\theta)$ calculer  les intervalles de confiance bootstrap $\mathcal I^*_{\text{norm}}$,  $\mathcal I^*_{\text{basic}}$,  $\mathcal I^*_{\text{perc}}$ et $\mathcal I^*_{\mathrm{BC_a}}$ pour $1/\mathbb E[X_1]$ en utilisant des répliques de $T=1/\bar X_n$ par le bootstrap **non paramétrique**.
- Mettre en oeuvre le calcul de l'intervalle studentisés par le bootstrap **non paramétrique**.
- Quant au  bootstrap **paramétrique** calculer les intervalles de confiance bootstrap $\mathcal I^*_{\text{norm}}$,  $\mathcal I^*_{\text{basic}}$ et  $\mathcal I^*_{\text{perc}}$.
-  Observer que **boot.ci** ne fournit pas  l'intervalle  $\mathcal I^*_{\mathrm{BC_a}}$. Au fait, **boot.ci** ne sait le calculer que dans le cas non paramétrique. Dans le cas paramétrique, la définition de l'intervalle est différente, mais elle n'est pas implementée dans **boot.ci**. Nous allons nous contenter des autres intervalles.
- Mettre en oeuvre le calcul de l'intervalle studentisé par le bootstrap **paramétrique**.

In [18]:
## données exponentielles
dataexp <- rexp(20,3)
## répliques bootstrap non paramétrique
T.exp.np <- boot(dataexp, function(obs, indice)1/mean(obs[indice]), R=1000)
T.exp.np
## IC bootstrap non paramétrique    
confint.np <- boot.ci(T.exp.np, type=c("norm", "basic", "perc", "bca"))
confint.np


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = dataexp, statistic = function(obs, indice) 1/mean(obs[indice]), 
    R = 1000)


Bootstrap Statistics :
    original    bias    std. error
t1*  2.27352 0.0367437    0.330739

BOOTSTRAP CONFIDENCE INTERVAL CALCULATIONS
Based on 1000 bootstrap replicates

CALL : 
boot.ci(boot.out = T.exp.np, type = c("norm", "basic", "perc", 
    "bca"))

Intervals : 
Level      Normal              Basic         
95%   ( 1.589,  2.885 )   ( 1.500,  2.806 )  

Level     Percentile            BCa          
95%   ( 1.741,  3.047 )   ( 1.685,  2.947 )  
Calculations and Intervals on Original Scale

In [19]:
## IC studentisé, bootstrap non paramétrique
boot.estim.var.np <- function(obs, indice){
    obs.boot <- obs[indice]
    estim.boot <- 1/mean(obs.boot) 
    var.boot <- var(replicate(50, 1/mean(sample(obs.boot, replace=T))))
    return(c(estim.boot, var.boot))
}

# répliques bootstrap avec estimation de la variance
T.exp.var <- boot(dataexp, boot.estim.var.np, R=1000)
# ICB studentisé
confint.np <- boot.ci(T.exp.var)
confint.np

BOOTSTRAP CONFIDENCE INTERVAL CALCULATIONS
Based on 1000 bootstrap replicates

CALL : 
boot.ci(boot.out = T.exp.var)

Intervals : 
Level      Normal              Basic             Studentized     
95%   ( 1.572,  2.885 )   ( 1.473,  2.804 )   ( 1.525,  2.914 )  

Level     Percentile            BCa          
95%   ( 1.743,  3.074 )   ( 1.720,  2.961 )  
Calculations and Intervals on Original Scale

In [20]:
## ICB paramétriques
boot.estim.param <- function(obs){
    n <- length(obs)
    estim.boot <- 1/mean(obs) 
    var.boot <- var(replicate(50, 1/mean(rexp(n, estim.boot))))
    return(c(estim.boot, var.boot))
}

T.exp.param <- boot(dataexp, boot.estim.param, R=1000, sim='parametric',
                    ran.gen=ran.gen.exp, mle=1/mean(dataexp))
boot.ci(T.exp.param, type=c('norm', 'basic', 'perc'))

BOOTSTRAP CONFIDENCE INTERVAL CALCULATIONS
Based on 1000 bootstrap replicates

CALL : 
boot.ci(boot.out = T.exp.param, type = c("norm", "basic", "perc"))

Intervals : 
Level      Normal              Basic              Percentile     
95%   ( 1.034,  3.253 )   ( 0.843,  3.012 )   ( 1.535,  3.704 )  
Calculations and Intervals on Original Scale

In [21]:
## ICB paramétrique type BCa : pas implementé dans boot.ci !
boot.ci(T.exp.param, type=c('bca'))

ERROR: Error in empinf(boot.out, index = index, t = t.o, ...): influence values cannot be found from a parametric bootstrap


In [22]:
## IC studentisé, bootstrap paramétrique
boot.estim.var.param <- function(obs){
    n <- length(obs)
    estim.boot <- 1/mean(obs) 
    var.boot <- var(replicate(50, 1/mean(rexp(n, estim.boot))))
    return(c(estim.boot, var.boot))
}

T.exp <- boot(dataexp, boot.estim.var.param, R=1000, sim='parametric',
              ran.gen=ran.gen.exp, mle=1/mean(dataexp))
T.exp
confint.param <- boot.ci(T.exp, type=c('norm', 'basic', 'stud', 'perc'))
confint.param


PARAMETRIC BOOTSTRAP


Call:
boot(data = dataexp, statistic = boot.estim.var.param, R = 1000, 
    sim = "parametric", ran.gen = ran.gen.exp, mle = 1/mean(dataexp))


Bootstrap Statistics :
     original     bias    std. error
t1* 2.2735205 0.12811093   0.5666144
t2* 0.3416567 0.02727892   0.2283335

BOOTSTRAP CONFIDENCE INTERVAL CALCULATIONS
Based on 1000 bootstrap replicates

CALL : 
boot.ci(boot.out = T.exp, type = c("norm", "basic", "stud", "perc"))

Intervals : 
Level      Normal              Basic         
95%   ( 1.035,  3.256 )   ( 0.772,  2.989 )  

Level    Studentized          Percentile     
95%   ( 1.276,  3.390 )   ( 1.558,  3.775 )  
Calculations and Intervals on Original Scale

In [23]:
all.confint <- list(confint.param[4:7], confint.np[4:7])
all.confint

[[1]]
[[1]]$normal
     conf                  
[1,] 0.95 1.034866 3.255953

[[1]]$basic
     conf                               
[1,] 0.95 975.98 25.03 0.771574 2.988976

[[1]]$student
     conf                               
[1,] 0.95 975.98 25.03 1.275501 3.389537

[[1]]$percent
     conf                               
[1,] 0.95 25.03 975.98 1.558065 3.775467


[[2]]
[[2]]$normal
     conf                  
[1,] 0.95 1.571598 2.885304

[[2]]$basic
     conf                               
[1,] 0.95 975.98 25.03 1.472767 2.803679

[[2]]$student
     conf                               
[1,] 0.95 975.98 25.03 1.524601 2.914239

[[2]]$percent
     conf                               
[1,] 0.95 25.03 975.98 1.743362 3.074274

## Question 3
Ecrire un programme pour simuler un grand nombre de jeu de données de loi exponentielle et calculer tous les  intervalles de confiance  $\mathcal I^*_{\text{norm}}$,  $\mathcal I^*_{\text{basic}}$,  $\mathcal I^*_{\text{stud}}$ et $\mathcal I^*_{\text{perc}}$  pour $1/\mathbb E[X]$  par les deux méthodes : bootstrap paramétrique et non paramétrique. Enfin estimer le niveau de confiance effectif de chaque intervalle. Commenter les résultats.

In [24]:
# theta = paramètre exponentiel
# n = taille d'échantillon
# M = nb de jeux de données simulés
# R = nb d'échantillons bootstrap
contains.theta <- function(vecIC, theta){
    L <- length(vecIC)
    is.theta.in <- (vecIC[L-1]<theta)&(vecIC[L]>theta)
    return(is.theta.in)
}

simul_icb_exp <- function(theta, n, M=100, R=100){
    prop <- matrix(0, 4, 2)
    colnames(prop) <- c('p','np')
    rownames(prop) <- c("norm", "basic", "stud", "perc")
    for (i in 1:M){
        obs <- rexp(n, theta)
        ## répliques bootstrap paramétriques avec estimations de variance
        T.param <- boot(obs, boot.estim.var.param, R, sim='parametric',
                        ran.gen=ran.gen.exp, mle=1/mean(obs))
        ## répliques bootstrap non paramétriques avec estimations de variance        
        T.np <- boot(obs, boot.estim.var.np, R)
        ## ICB paramétrique
        confint.param <- boot.ci(T.param, type=c('norm', 'basic', 'stud', 'perc'))
        ## ICB non paramétrique
        confint.np <- boot.ci(T.np,type=c("norm", "basic", "stud", "perc"))
        ## theta inclus dans ICB ?
        prop[,1] <- prop[,1] + sapply(confint.param[4:7], function(x) contains.theta(x, theta))
        prop[,2] <- prop[,2] + sapply(confint.np[4:7], function(x) contains.theta(x, theta))
    }
    return(prop/M)
}

In [25]:
simul_icb_exp(2, 10)

,p,np
norm,0.98,0.92
basic,0.93,0.90
stud,0.93,0.95
perc,0.92,0.86


## Question 4

On veut étudier ce qui se passe si les données ne suivent pas exactement une loi exponentielle. Maintenant les données  suivent la loi Gamma $\Gamma(\alpha,\theta)$.
La quantité inconnue est toujours $1/\mathbb E[X]=\theta/\alpha$, estimée par $T=1/\bar X_n$.

Modifier votre programme pour générer des données de loi Gamma. Ensuite, appliquer exactement les mêmes calculs comme si les données avaient une loi exponentielle (notamment l'échantillons bootstrap paramétrique est toujours tiré selon une loi exponentielle). 

Rappelons que $\mathcal E(\theta)=\Gamma(1,\theta)$.  Comment évolue les différent niveaux de confiance effectif lorsque les  données suivent une loi $\Gamma(\alpha,\theta)$ avec des valeurs de $\alpha$ inférieures à  1 ?



In [26]:
# gam : $alpha, $theta : liste de paramètres de la loi Gamma

simul_icb_gamma <- function(gam, n, M=100, R=100){
    prop <- matrix(0, 4, 2)
    colnames(prop) <- c('p','np')
    rownames(prop) <- c("norm", "basic", "stud", "perc")
    for (i in 1:M){
        obs <- rgamma(n, gam$alpha, gam$theta)
        ## répliques bootstrap paramétriques avec estimations de variance
        T.param <- boot(obs, boot.estim.var.param, R, sim='parametric',
                        ran.gen=ran.gen.exp, mle=1/mean(obs))
        ## répliques bootstrap non paramétriques avec estimations de variance        
        T.np <- boot(obs, boot.estim.var.np, R)
        ## ICB paramétrique
        confint.param <- boot.ci(T.param, type=c('norm', 'basic', 'stud', 'perc'))
        ## ICB non paramétrique
        confint.np <- boot.ci(T.np, type=c("norm", "basic", "stud", "perc"))
        ## theta inclus dans ICB ?
        prop[,1] <- prop[,1] + sapply(confint.param[4:7], function(x) contains.theta(x, gam$theta/gam$alpha))
        prop[,2] <- prop[,2] + sapply(confint.np[4:7], function(x) contains.theta(x, gam$theta/gam$alpha))
    }
    return(prop/M)
}

In [27]:
simul_icb_gamma(list(alpha=.8, theta=2), 10)

,p,np
norm,0.96,0.93
basic,0.89,0.86
stud,0.95,0.93
perc,0.89,0.86


In [28]:
simul_icb_gamma(list(alpha=.5, theta=2), 10)

,p,np
norm,0.94,0.94
basic,0.80,0.85
stud,0.91,0.92
perc,0.85,0.85


In [29]:
simul_icb_gamma(list(alpha=2, theta=2), 10)

,p,np
norm,1.00,0.95
basic,0.97,0.90
stud,0.99,0.97
perc,0.98,0.92


In [30]:
simul_icb_gamma(list(alpha=.1, theta=2), 10)

,p,np
norm,0.60,0.97
basic,0.65,0.75
stud,0.45,0.96
perc,0.44,0.62
